# Simplified LoRA adaptation of FFN

We will show how to do LoRA on a simple FFN by first pre-training it on Fashion MNIST and then finetune it on MNIST. As those datasets don't have a ton to do the performance will be quite bad, but we seek to show how to do PEFT in general regardless of the model

## Pre-Training

In [ ]:
pip install datasets

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.datasets import fashion_mnist

# Load Fashion MNIST dataset
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

# Normalize the images
train_images = train_images / 255.0
test_images = test_images / 255.0

# Define the more complex model
model = keras.Sequential([
    Flatten(input_shape=(28, 28)),

    Dense(1024, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),

    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),

    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),

    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),

    Dense(64, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),

    Dense(10, activation='softmax')  # 10 classes in Fashion MNIST
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])




In [ ]:
model.summary()

In [ ]:
# Train the model
model.fit(train_images, train_labels, epochs=15, validation_split=0.2)

## Lora-Adaptation

Load the new dataset

In [ ]:
from tensorflow.keras.datasets import mnist

# Load MNIST dataset
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize the images
train_images = train_images / 255.0
test_images = test_images / 255.0

# Reshape images for the model
train_images = train_images.reshape((-1, 28, 28, 1))
test_images = test_images.reshape((-1, 28, 28, 1))


In [ ]:
class LoraLayer(keras.layers.Layer):
    def __init__(
        self,
        original_layer,
        rank=8,
        num_heads =1,
        dim = 1,
        trainable=False,
        **kwargs,
    ):
        # We want to keep the name of this layer the same as the original
        # dense layer.
        original_layer_config = original_layer.get_config()
        name = original_layer_config["name"]

        kwargs.pop("name", None)

        super().__init__(name=name, trainable=trainable, **kwargs)

        self.rank = rank


        # Layers.

        # Original dense layer.
        self.original_layer = original_layer
        # No matter whether we are training the model or are in inference mode,
        # this layer should be frozen.
        self.original_layer.trainable = False

        # LoRA dense layers.
        self.A = keras.layers.Dense(
            units=rank,
            use_bias=False,
            trainable=trainable,
            name=f"lora_A",
        )

        self.B = keras.layers.Dense(
            units=dim,
            use_bias=False,
            trainable=trainable,
            name=f"lora_B",
        )

    def call(self, inputs):
        original_output = self.original_layer(inputs)
        if self.trainable:
            # If we are fine-tuning the model, we will add LoRA layers' output
            # to the original layer's output.
            lora_output = self.B(self.A(inputs))
            return original_output + lora_output

        # If we are in inference mode, we "merge" the LoRA layers' weights into
        # the original layer's weights - more on this in the text generation
        # section!
        return original_output

We will randomly change some Dense layers into Lora Adapted layers

In [ ]:
import random
# Define a function to replace dense layers with LoraLayer
def replace_with_lora(model):
    new_model = keras.Sequential()
    for layer in model.layers:
        if isinstance(layer, Dense) and random.random() > 0.5:
            new_model.add(LoraLayer(original_layer=layer, rank=4, dim=layer.units, trainable=True))
        else:
            new_model.add(layer)
    return new_model

# Replace layers in the model
lora_model = replace_with_lora(model)

lora_model.build(input_shape=(None, 28, 28, 1))

# Compile the model
lora_model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-4),
                   loss='sparse_categorical_crossentropy',
                   metrics=['accuracy'])


In [ ]:
lora_model.summary()

Notice the non-trainable parameters

In [ ]:
# Fine-tune the model
lora_model.fit(train_images, train_labels, epochs=10, validation_split=0.2)

As mentioned, performance sucks, but the important thing is that we finetuned only the LoraLayers